# IGC Take Home

In [1]:
import pandas as pd
import numpy as np
import glob
import os

## Identified Data Sources

1. IEA:
    - Total Energy Supply
    - Total Domestic Energy Supply
    - Total Energy Supply Imported
    - Total Energy Consumption
2. IRENA:
    - Renewable Energy Capacity
3. LEI:
    - Consumption Patterns by region
3. World Bank
    - Electricity Production
    - Energy Dependency 
4. Karandaaz
    - Fuel Prices

## Key Metrics:
- Renewable Share of Total Generation
- Year-on-Year change in Production
- Renewable Capacity Growth
- Average Electricity Price
- Average Fuel Price
- Tracking Energy Reform
- Consumption Patterns

# Loading Relevant Data

In [2]:
final_column_order = ['year', 'metric', 'category', 'value', 'units', 'source']

### IEA

In [3]:
# iea data path
iea_path = "../data/raw/iea/"
csv_files = glob.glob(os.path.join(iea_path, '*.csv'))

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    df.rename(columns=({df.columns[0]: "category"}), inplace=True)
    df['source_file'] = os.path.basename(file).replace(".csv", "")
    dfs.append(df)

# Concatenating all dataframes into a single dataframe
df_all_iea = pd.concat(dfs, ignore_index=True).rename(columns={"source_file": "metric"})

df_all_iea['metric'] = df_all_iea['metric'].str.replace("_", " ").str.title()
df_all_iea['source'] = 'IEA'
df_all_iea.columns = df_all_iea.columns.str.lower()

df_all_iea = df_all_iea[final_column_order]
df_all_iea.head()


,year,metric,category,value,units,source
0,2000,Total Energy Consumption,Industry,468001.0,TJ,IEA
1,2001,Total Energy Consumption,Industry,478500.0,TJ,IEA
2,2002,Total Energy Consumption,Industry,516926.0,TJ,IEA
3,2003,Total Energy Consumption,Industry,591680.0,TJ,IEA
4,2004,Total Energy Consumption,Industry,660045.0,TJ,IEA


## IRENA

In [4]:
irena_path = "../data/raw/irena/irena_stats_2024.xlsx"
sheet_name = "All data"

df_irena = pd.read_excel(irena_path, sheet_name=sheet_name)
df_irena = df_irena[df_irena['Country'] == 'Pakistan']


In [5]:
cols = ['Year', 'RE or Non-RE', 'Technology', 'Producer Type', 'Electricity Generation (GWh)', 'Electricity Installed Capacity (MW)']
df_irena_filt = df_irena[cols]

In [6]:
df_irena_long = df_irena_filt.melt(
    id_vars=['Year', 'Technology'],
    value_vars=['Electricity Generation (GWh)', 'Electricity Installed Capacity (MW)'],
    var_name='metric',
    value_name='value'
)
df_irena_long['units'] = df_irena_long['metric'].str.split(" ").str[-1].str.replace("(", "").str.replace(")", "")
df_irena_long['source'] = 'IRENA'
df_irena_long.columns = ['year', 'category', 'metric', 'value', 'units', 'source']
df_irena_long = df_irena_long[final_column_order]
df_irena_long.head()

,year,metric,category,value,units,source
0,2014,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
1,2016,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
2,2020,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
3,2022,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
4,2023,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA


## LEI

In [7]:
lei_rewdp_path = "../data/raw/lei/rewdp_dataset"
csv_files = glob.glob(os.path.join(lei_rewdp_path, "**/*.csv"), recursive=True)


lei_dfs = []
for file in csv_files:
    usecols=[col for col in pd.read_csv(file, nrows=0).columns if "usage" in col.lower()]
    usecols.append('datetime')    
    df = pd.read_csv(file, usecols=usecols)
    df['source_file'] = os.path.basename(file)
    df['city'] = file.split("/")[-2]
    lei_dfs.append(df)

# # Concatenate all sampled dataframes
df_all_lei_rewdp = pd.concat(lei_dfs, ignore_index=True)
df_all_lei_rewdp.head()

,datetime,Usage (kW),source_file,city,Usage_kW,Usage..kW.
0,2023-11-01,0.944150,lahore_House8.csv,Lahore,NaN,NaN
1,2023-11-01 00:01:00,0.948467,lahore_House8.csv,Lahore,NaN,NaN
2,2023-11-01 00:02:00,0.947617,lahore_House8.csv,Lahore,NaN,NaN
3,2023-11-01 00:03:00,0.947733,lahore_House8.csv,Lahore,NaN,NaN
4,2023-11-01 00:04:00,0.953583,lahore_House8.csv,Lahore,NaN,NaN


In [8]:
df_all_lei_rewdp

,datetime,Usage (kW),source_file,city,Usage_kW,Usage..kW.
0,2023-11-01,0.944150,lahore_House8.csv,Lahore,NaN,NaN
1,2023-11-01 00:01:00,0.948467,lahore_House8.csv,Lahore,NaN,NaN
2,2023-11-01 00:02:00,0.947617,lahore_House8.csv,Lahore,NaN,NaN
3,2023-11-01 00:03:00,0.947733,lahore_House8.csv,Lahore,NaN,NaN
4,2023-11-01 00:04:00,0.953583,lahore_House8.csv,Lahore,NaN,NaN
...,...,...,...,...,...,...
35734526,2024-10-30 23:55:00,0.512317,House26.csv,Peshawar,NaN,NaN
35734527,2024-10-30 23:56:00,0.461567,House26.csv,Peshawar,NaN,NaN
35734528,2024-10-30 23:57:00,0.433567,House26.csv,Peshawar,NaN,NaN
35734529,2024-10-30 23:58:00,0.261067,House26.csv,Peshawar,NaN,NaN


In [9]:
df_all_lei_rewdp['usage_kw'] = df_all_lei_rewdp[['Usage (kW)', 'Usage_kW', 'Usage..kW.']].bfill(axis=1).iloc[:, 0]
df_all_lei_rewdp = df_all_lei_rewdp[['datetime', 'usage_kw', 'city', 'source_file']]
df_all_lei_rewdp.head()


,datetime,usage_kw,city,source_file
0,2023-11-01,0.944150,Lahore,lahore_House8.csv
1,2023-11-01 00:01:00,0.948467,Lahore,lahore_House8.csv
2,2023-11-01 00:02:00,0.947617,Lahore,lahore_House8.csv
3,2023-11-01 00:03:00,0.947733,Lahore,lahore_House8.csv
4,2023-11-01 00:04:00,0.953583,Lahore,lahore_House8.csv


In [10]:
df_avg_usg_city = df_all_lei_rewdp.groupby('city')['usage_kw'].median().reset_index()
df_avg_usg_city

,city,usage_kw
0,Islamabad,0.278117
1,Karachi,0.579233
2,Lahore,0.599120
3,Multan,0.439950
4,Peshawar,0.371200
5,Skardu,0.004083


In [11]:
weather_path = "../data/raw/lei/weather_dataset"
csv_files = glob.glob(os.path.join(weather_path, "*.csv"))

weather_dfs = []
for file in csv_files:
    df = pd.read_csv(file, usecols=['datetime', 'Wind Speed', 'Wind Direction', 'Solar Radiation', 'Solar Energy'])
    df['source_file'] = os.path.basename(file)
    df['city'] = df['source_file'].str.split(".csv").str[0]
    weather_dfs.append(df)

df_all_weather = pd.concat(weather_dfs, ignore_index=True)
df_all_weather.head()

,datetime,Wind Speed,Wind Direction,Solar Radiation,Solar Energy,source_file,city
0,2023-07-01,0.0,0.0,0.0,0.0,Multan.csv,Multan
1,2023-07-01 01:00:00,11.2,230.0,0.0,0.0,Multan.csv,Multan
2,2023-07-01 02:00:00,7.2,180.0,0.0,0.0,Multan.csv,Multan
3,2023-07-01 03:00:00,11.2,230.0,0.0,0.0,Multan.csv,Multan
4,2023-07-01 04:00:00,0.0,0.0,0.0,0.0,Multan.csv,Multan


In [12]:
df_all_weather_avg = df_all_weather.groupby('city')[['Wind Speed', 'Solar Radiation', 'Solar Energy']].median().reset_index()

# Karandaaz

In [13]:
kdz_path = "../data/raw/kdz/Petrol Price.csv"
df_kdz = pd.read_csv(kdz_path)

# dropping first row
df_kdz = df_kdz.loc[1:]
df_kdz.head()
df_kdz['year'] = pd.to_datetime(df_kdz['Period End']).dt.year
df_kdz['Pakistan'] = df_kdz['Pakistan'].astype(float)
df_petrol_prices = df_kdz.groupby('year')['Pakistan'].median().reset_index()
df_petrol_prices['metric'] = 'Petrol Price'
df_petrol_prices['units'] = 'PKR'
df_petrol_prices.rename(columns={'Pakistan': 'value'}, inplace=True)
df_petrol_prices['category'] = "Missing"
df_petrol_prices['source'] = 'Karandaaz'
df_petrol_prices = df_petrol_prices[final_column_order]
df_petrol_prices.head()

,year,metric,category,value,units,source
0,2013,Petrol Price,Missing,104.27,PKR,Karandaaz
1,2014,Petrol Price,Missing,109.86,PKR,Karandaaz
2,2015,Petrol Price,Missing,75.09,PKR,Karandaaz
3,2016,Petrol Price,Missing,65.12,PKR,Karandaaz
4,2017,Petrol Price,Missing,73.31,PKR,Karandaaz


## World Bank

In [14]:
wb_path = "../data/raw/wb/wb_electricity_data.csv"
df_access = pd.read_csv(wb_path)

df_access_long = df_access.drop(columns=['Series Code']).melt(
    id_vars=['Country Name', 'Country Code', 'Series Name'],
    var_name='year',
    value_name='value'
)
# Clean up year column to just the year number
df_access_long['year'] = df_access_long['year'].str.split(" ").str[0].astype(int)
df_access_long = df_access_long[df_access_long['Series Name'].str.contains("Access to electricity", na=False)]

df_access_long.head()
df_access_long.rename(columns={'Series Name': "metric"}, inplace=True)
df_access_long['units'] = 'Percentage'
df_access_long['category'] = "Missing"
df_access_long['source'] = 'World Bank'
df_access_long = df_access_long[final_column_order]


## Combining Datasets

## Key Metrics

In [15]:
df_combined = pd.concat([df_all_iea, df_irena_long, df_access_long])

In [16]:
df_combined.head()

,year,metric,category,value,units,source
0,2000,Total Energy Consumption,Industry,468001.0,TJ,IEA
1,2001,Total Energy Consumption,Industry,478500.0,TJ,IEA
2,2002,Total Energy Consumption,Industry,516926.0,TJ,IEA
3,2003,Total Energy Consumption,Industry,591680.0,TJ,IEA
4,2004,Total Energy Consumption,Industry,660045.0,TJ,IEA


## Consumption Patterns

In [17]:
df_consump_patterns_geo = df_avg_usg_city.merge(df_all_weather_avg, on='city')
df_consump_patterns_geo

,city,usage_kw,Wind Speed,Solar Radiation,Solar Energy
0,Islamabad,0.278117,6.50,7.10,0.0
1,Karachi,0.579233,13.30,15.25,0.1
2,Lahore,0.599120,1.55,11.35,0.0
3,Multan,0.439950,0.00,12.00,0.0
4,Peshawar,0.371200,7.60,11.30,0.0
5,Skardu,0.004083,3.60,11.15,0.0


In [18]:
df_avg_usg_city.head()

,city,usage_kw
0,Islamabad,0.278117
1,Karachi,0.579233
2,Lahore,0.599120
3,Multan,0.439950
4,Peshawar,0.371200


# Calculating Metrics

- Renewable Share of Total Generation
- Year-on-Year change in Production
- Renewable Capacity Growth
- Average Fuel Price
- Consumption Patterns by Location

In [19]:
source = "Total Energy Supply"
df_sup = df_all_iea[df_all_iea['metric'] == source]

In [20]:
source = "Total Domestic Energy Supply"
df_dom_sup = df_all_iea[df_all_iea['metric'] == source]
df_dom_sup.head()

,year,metric,category,value,units,source
216,2000,Total Domestic Energy Supply,Coal and coal products,51785.0,TJ,IEA
217,2001,Total Domestic Energy Supply,Coal and coal products,55703.0,TJ,IEA
218,2002,Total Domestic Energy Supply,Coal and coal products,55063.0,TJ,IEA
219,2003,Total Domestic Energy Supply,Coal and coal products,55307.0,TJ,IEA
220,2004,Total Domestic Energy Supply,Coal and coal products,72461.0,TJ,IEA


In [21]:
df_combined['value'] = pd.to_numeric(df_combined['value'], errors='coerce')

def safe_pct_change(series):
    prev = series.shift(1)
    mask = (prev != 0) & (~prev.isna())
    pct = np.full(series.shape, np.nan)
    pct[mask] = (series[mask] - prev[mask]) / prev[mask] * 100
    return pct

df_combined = df_combined.sort_values(['metric', 'category', 'year'])
df_combined['yoy_change_pct'] = df_combined.groupby(['metric', 'category'])['value'].transform(safe_pct_change)
df_combined['moving_avg_3yr'] = df_combined.groupby(['metric', 'category'])['value'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df_combined.head()

,year,metric,category,value,units,source,yoy_change_pct,moving_avg_3yr
0,2000,Access to electricity (% of population),Missing,72.8,Percentage,World Bank,NaN,72.800000
17,2001,Access to electricity (% of population),Missing,73.9,Percentage,World Bank,1.510989,73.350000
34,2002,Access to electricity (% of population),Missing,74.9,Percentage,World Bank,1.353180,73.866667
51,2003,Access to electricity (% of population),Missing,76.0,Percentage,World Bank,1.468625,74.933333
68,2004,Access to electricity (% of population),Missing,77.0,Percentage,World Bank,1.315789,75.966667


## Saving

In [22]:
key_metrics_path = "../data/clean/key_metrics.csv"
df_combined.to_csv(key_metrics_path, index=False)

In [24]:
consumption_patterns_path = "../data/clean/consumption_patterns.csv"
df_consump_patterns_geo.to_csv(consumption_patterns_path, index=False)

In [25]:
df_irena_long

,year,metric,category,value,units,source
0,2014,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
1,2016,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
2,2020,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
3,2022,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
4,2023,Electricity Generation (GWh),Coal and peat,NaN,GWh,IRENA
...,...,...,...,...,...,...
941,2019,Electricity Installed Capacity (MW),Onshore wind energy,1235.7,MW,IRENA
942,2020,Electricity Installed Capacity (MW),Onshore wind energy,1235.7,MW,IRENA
943,2021,Electricity Installed Capacity (MW),Onshore wind energy,1335.0,MW,IRENA
944,2022,Electricity Installed Capacity (MW),Onshore wind energy,1845.0,MW,IRENA


In [27]:
df_petrol_prices['yoy_change_pct'] = df_petrol_prices.groupby(['metric', 'category'])['value'].transform(safe_pct_change)


In [30]:
df_petrol_prices['yoy_change_pct'].mean()

11.390474851304857

In [31]:
df_consump_patterns_geo

,city,usage_kw,Wind Speed,Solar Radiation,Solar Energy
0,Islamabad,0.278117,6.50,7.10,0.0
1,Karachi,0.579233,13.30,15.25,0.1
2,Lahore,0.599120,1.55,11.35,0.0
3,Multan,0.439950,0.00,12.00,0.0
4,Peshawar,0.371200,7.60,11.30,0.0
5,Skardu,0.004083,3.60,11.15,0.0
